In [ ]:
DIR="../inputs/RootFiles/"
import ROOT
%jsroot on
canvas = ROOT.TCanvas("c1", "", 200, 10, 1000, 1000)
ROOT.gStyle.SetOptStat(0)
ROOT.gStyle.SetOptTitle(0)

# More functions... 

In [ ]:
lumi = {12:11580. , 11:2330.}


def get_histogram(file_dir, key, **hist_args):
    with ROOT.TFile.Open(f"{DIR}/{file_dir}") as file:
        hist = file.demo.Get(key)
        hist.SetDirectory(0)
        
    if "title" in hist_args:
        title = hist_args["title"]
        hist.SetTitle(title)
    
    if "scale" in hist_args:
        scale = hist_args["scale"]
        hist.Scale(scale)
        
    return hist


def modify_histogram(hist, **hist_args):

    if "title" in hist_args:
        title = hist_args["title"]
        hist.SetTitle(title)
    
    if "scale" in hist_args:
        scale = hist_args["scale"]
        hist.Scale(scale)


def histogram_stack(name, historams, last=False):
    stack = ROOT.THStack(name , "")
    for hist in historams:
        stack.Add(hist)

    return stack


def ZZto4lep_hists():
    sfZZ = 1.386
    zz_xsections = {12:{'4e':0.077 , '4mu':0.077 , '2mu2e':0.18},
                    11:{'4e':0.067 , '4mu':0.067 , '2mu2e':0.15}}

    zz_totals = {12:{'4mu':1499064 , '4e':1499093 , '2mu2e':1497445},
                11:{'4mu':1447136 , '4e':1493308 , '2mu2e':1479879}}

    hists = []
    for decaymode in ['4mu' , '4e' , '2mu2e']:
        for year in [11,12]:

            file_directory  = f"20{year}/simulation/ZZ{decaymode}{year}.root"
            file_key        = f"mass{decaymode}_8TeV_low"
            scale_factor    = zz_xsections[year][decaymode]*lumi[year]*sfZZ / zz_totals[year][decaymode] 
            histogram_title = f"{decaymode.replace('mu' , '#mu')}, 20{year}"

            h = get_histogram(file_dir = file_directory,
                              key      = file_key,
                              scale    = scale_factor,
                              title    = histogram_title)
            hists.append(h)

    return hists

def TTbarto4lep_hists():
    tt_xsections = {12:200. , 11:177.31}
    tt_totals    = {12:6423106 , 11:9771205}
    sfTTBar      = {11:0.11 , 12:1 }

    hists = []

    for year in [11,12]:
        file_directory = f"20{year}/simulation/TTBar{year}.root"

        for decaymode in ['4mu' , '4e' , '2mu2e']:
            file_key          = f"mass{decaymode}_8TeV_low"
            scale_factor = tt_xsections[year]*lumi[year]*sfTTBar[year] / tt_totals[year]
            histogram_title        = "{0}, 20{1}".format( decaymode.replace('mu' , '#mu') ,year)

            current_hist = get_histogram(file_dir = file_directory,
                              key      = file_key,
                              scale    = scale_factor,
                              title    = histogram_title)

            hists.append(current_hist)

    return hists


def DrellYan_hists():
    dy_xsections = {12:{10:10.742 , 50:2955.} ,
                   11:{10:9507. , 50:2475.}}
    dy_totals    = {12:{10:6462290 , 50:29426492},
                    11:{10:39909640, 50:36408225}}
    
    sfDY = 1.12
    
    hists = []
    for mass in [10,50]:
        fname_extension = 'TuneZ' if mass == 50 else ''
        for year in [11,12]:
            file_directory = f"20{year}/simulation/DY{mass}{fname_extension}{year}.root"

            for decaymode in ['4mu' , '4e' , '2mu2e']:
                file_key = f"mass{decaymode}_8TeV_low"
                scale_factor = dy_xsections[year][mass]*lumi[year]*sfDY / dy_totals[year][mass]
                histogram_title = "DY({2}), {0}, 20{1}".format( decaymode.replace('mu' , '#mu') ,year , mass)

                current_hist = get_histogram(file_dir = file_directory,
                                             key      = file_key,
                                             scale    = scale_factor,
                                             title    = histogram_title)

                hists.append(current_hist)

    return hists
    
    

<div dir="rtl">
  <p>
    در این تمرین میخواهیم نمودار داده ها که در بخش اول به دست اورده ایم با مجموع شبیه سازی هایی که تا به حال انجام داده ایم مقایسه کنیم و ببینیم آیا در داده ها هیگز وجود دارد؟
  </p>
  <p>
    از نظر تکنیگی، به سادگی میتوان اخرین و بالاترین هیستوگرام موجود در THStack 
    را به عنوان جمع همه ی هیستوگرام ها داشت. برای این منظور میتوان از متد زیر استفاده کرد:
      
</div>

`GetStack().Last()`

<div dir="rtl">
  <p>
    به تنظیماتی که برای تغییر نحوه ی نمایش نمودار ها اعمال شده دقت کنید.
  </p>
</div>

# Histogram of the `Data!`

In [ ]:
ROOT.gStyle.SetPalette(ROOT.kRainBow)
allHists = []
for lep in ['Mu' , 'E']:
    for year in [11,12]:
        file_directory  = "20{0}/data/Double{1}{0}.root".format(year , lep)
        file_key        = "mass4{0}_8TeV_low".format(lep.lower())
        histogram_title = "4{0}, 20{1}".format( '#mu' if lep=="Mu" else 'e',year)
        
        h = get_histogram(file_dir = file_directory,
                           key      = file_key,
                           title    = histogram_title)
        allHists.append(h)
            
        if lep=='Mu': #if the doubleMu file is opened, 2mu2e histogram should also be read
            file_key        = "mass2mu2e_8TeV_low"
            histogram_title = "2#mu+2e, 20{0}".format(year)
            h = get_histogram(file_dir = file_directory,
                              key      = file_key,
                              title    = histogram_title)
            allHists.append(h)

Data_4l_stack = histogram_stack("Data4lep", allHists)
Data_4l = Data_4l_stack.GetStack().Last()

# Formatting 
Data_4l.SetMarkerColor(ROOT.kBlack)
Data_4l.SetMarkerStyle(20)
Data_4l.SetMarkerSize(1.7)
Data_4l.SetLineColor(ROOT.kBlack)
Data_4l.SetLineWidth(1)
Data_4l.Draw("PE1")

# Drawing
canvas.Draw()

# Extracting the background events histograms...

## Histogram of $ZZ \rightarrow 4l$ Events

In [ ]:
ROOT.gStyle.SetPalette(ROOT.kRainBow)
ZZto4l_stack = histogram_stack("ZZto4lep", ZZto4lep_hists())

ZZto4l = ZZto4l_stack.GetStack().Last()

ZZto4l.SetLineColor(ROOT.kBlack)
ZZto4l.SetFillColor(ROOT.kAzure -9)
ZZto4l.SetLineWidth(2)
ZZto4l.SetLineStyle(1)
ZZto4l.Draw("hist")

canvas.Draw()

<div dir="rtl">
  <p>
این نمودارها نشان دهنده ی مجموع تمام داده ها و مقادیر شبیه سازی شده برای
    ZZ
    نمایش داده شده. همین کار را برای ttbar و dy تکرار میکنیم:
  </p>
</div>

## Histogram of $t\bar{t}$ Events 

In [ ]:
TTbarto4l_stack = histogram_stack("TTbarto4l", TTbarto4lep_hists())

TTbarto4l = TTbarto4l_stack.GetStack().Last()
TTbarto4l.SetLineColor(ROOT.kBlack)
TTbarto4l.SetFillColor(ROOT.kGray)
TTbarto4l.SetLineWidth(2)
TTbarto4l.SetLineStyle(1)

## Histogram of Drell-Yan Events

In [ ]:
DYto4l_stack = histogra_stack("DYto4lep", DrellYan_hists())

DYto4l = DYto4l_stack.GetStack().Last()
DYto4l.SetLineColor(ROOT.kBlack)
DYto4l.SetFillColor(ROOT.kGreen -5)
DYto4l.SetLineWidth(2)
DYto4l.SetLineStyle(1)

<div dir="rtl">
  <p>
با داشتن هیستوگرام تمام رویدادهای شبیه سازی شده، میتوان جمع شبیه سازی را با داده های اصلی مقایسه کرد.
    کافی است هیستوگرامهای شبیه سازی را داخل stack قرار دهیم
    و در کنار داده ی اصلی بکشیم. 
  </p>
  <p>
    در کد زیر، علاوه بر این کار یک legend هم ساخته شده
    و در عکس نمایش داده میشود.
  </p>
</div>

# Compare Simulation with Data, Visually...

In [ ]:
simulation = histogram_stack("simulation", [DYto4l, TTbarto4l, ZZto4l])

leg = ROOT.TLegend(.62, .70, .82, .88)
leg.SetFillColor(0)
leg.SetBorderSize(0)
leg.SetTextFont(42)
leg.SetTextSize(0.038)

leg.AddEntry(Data_4l, "Data", "PE1")
leg.AddEntry(DYto4l, "Z/#gamma* + X", "f")
leg.AddEntry(TTbarto4l, "TTBar", "f")
leg.AddEntry(ZZto4l, "ZZ -> 4l", "f")

simulation.Draw("hist")
Data_4l.Draw("PE1 same")

simulation.GetXaxis().SetTitle("m_{4l} (GeV)")
simulation.GetYaxis().SetTitle("Events / 3 GeV")
simulation.GetYaxis().SetTitleOffset(1.2)
simulation.SetMaximum(30)

txt1 = ROOT.TLatex()
txt1.DrawTextNDC(0.15,0.84,"CMS Open Data")
txt1.SetTextSize(0.03)
txt1.SetNDC()
txt1.SetTextFont(42)
txt1.DrawLatex(0.14, 0.91, "#sqrt{s} = 7 TeV, L = 2.3 fb^{-1}, #sqrt{s} = 8 TeV, L = 11.6 fb^{-1}" )

leg.Draw()
canvas.Draw()

<div dir="rtl">
  <p>
    به نظر میاد شبیه سازی تا حد خیلی خوبی میتونه داده ها رو توصیف کنه.
    مخصوصا در نزدیک جرم بوزون 
    Z.
  </p>
  <p>
    مقداری اختلاف بین داده و شبیه سازی در نزدیکی جرم ۱۲۵ گیگا الکترون ولت، و همچنین جرم ۱۴۵ وجود دارد.
  </p>
  <p>
    <ul>
      <li> سیگنیفیکنس اختلاف داده با شبیه سازی در این دو جرم چقدر است ؟
      </li>
  </ul>
  </p>
</div>

# Statistical Significance of Oservations

<div dir="rtl">
<p>
  حال پیش بینی استاندارد مدل برای تولید هیگز و واپاشی آن به چهار لپتون را شبیه سازی میکنیم. برای خواندن نتیجه ی شبیه سازی کافی است داده ها را از فایلهای مناسب بخوانیم و رسم کنیم:
</p>
</div>

# Say 'Higgs!'

In [ ]:
scalexsecHZZ = {12:0.0065 , 11:0.0057}
nevtHZZ = {12:299973 , 11:299683 }

HzzPlot = ROOT.THStack("HzzPlot" , "")

for y in [11,12]:
    f = ROOT.TFile.Open("{0}20{1}/simulation/HZZ{1}.root".format(DIR , year ) )
    for decaymode in ['4mu' , '4e' , '2mu2e']:
        h = f.Get("demo/mass{0}_8TeV_low".format(decaymode))
        h.SetDirectory(0)
        h.SetTitle("{0}, 20{1}".format(decaymode , y))
        h.Scale( lumi[y]*scalexsecHZZ[y]/nevtHZZ[y])
        HzzPlot.Add( h )
        
HZZto4l = HzzPlot.GetStack().Last()
HZZto4l.SetLineColor(ROOT.kRed)
HZZto4l.SetLineWidth(2)
HZZto4l.SetLineStyle(1)
HZZto4l.SetFillStyle(0)
HZZto4l.SetFillColor(0)

HzzPlot.Draw("pfc plc pmc HIST")
HzzPlot.GetXaxis().SetTitle('m_{4l} [GeV]')
canvas.BuildLegend(0.7,0.7,0.95,0.95 , "Invariant mass of 4 leptons, pp -> H", "f")
canvas.Draw()

<div dir="rtl">
  <p>
نمودار بالا نشان میدهند اگر هیگز با جرم ۱۲۵ گیگا الکترون ولت تولید شود، میتوان انتظار داشت ۵ تا از آن رویداد ها را در خانه ی مرکزی هیستوگرام نشان داده شده مشاهده کنیم
  </p>

</div>

In [ ]:
if not HiggsAdded:
    leg.AddEntry(HZZto4l, "m_{H} = 125 GeV", "f")
    simulation.Add(HZZto4l)
    HiggsAdded = True

simulation.Draw("hist")
Data_4l.Draw("PE1 same")

simulation.GetXaxis().SetTitle("m_{4l} (GeV)")
simulation.GetYaxis().SetTitle("Events / 3 GeV")
simulation.GetYaxis().SetTitleOffset(1.2)
simulation.SetMaximum(30)

txt1.DrawTextNDC(0.15,0.84,"CMS Open Data")
txt1.SetTextSize(0.03)
txt1.SetNDC()
txt1.SetTextFont(42)
txt1.DrawLatex(0.14, 0.91, "#sqrt{s} = 7 TeV, L = 2.3 fb^{-1}, #sqrt{s} = 8 TeV, L = 11.6 fb^{-1}" )

leg.Draw()
canvas.Draw()

<div dir="rtl">
 <p>
   به وضوح معلوم است که این سیگنال میتواند اختلاف پس زمینه با داده ها را توصیف کند.
  </p>
  <h1>
    باقی تمرین‌ها
  </h1>
  <ul>
    <li>
      <h2>
        تمرین ۲
      </h2>
      <p>
        نمودار جمع سیگنال و پس زمینه را از stack 
        استخراج کنید. با مقایسه ی آن با داده میتوان مقدار 
        χ2
        را به راحتی میتوان از این مقایسه استخراج کرد. کافی است از متد 
        <code><a href="https://root.cern.ch/doc/master/classTH1.html#a6c281eebc0c0a848e7a0d620425090a5">Chi2Test()</a></code>
        هیستوگرام استفاده کنید.
      </p>
      <p>
         مقدار 
        χ2
        هیستوگرام ها را گزارش کنید.
      </p>
    </li>
        <li>
      <h2>
تمرین ۳
          </h2>
      <p>
        در مقایسه ی نمودار شبیه سازی با داده میتوان یک پارامتر اضافه کرد. میتوانیم قبل از اضافه کردن سیگنال به پس زمینه ها، سیگنال را در یک عدد که میتوانیم آن را «شدت سیگنال» بنامیم ضرب کنیم. دقت کنید که قبلا با دستور 
        <code>Scale</code>
        از هیستوگرام آشنا شده اید. این کمیت را با μ معمولا نشان میدهیم.
      </p>
          <p>
            مقدار μ 
            را بین ۰.۹ تا ۱.۱ در ۲۰ بازه تغییر دهید و برای هر کدام مقدار 
            χ2
            را محاسبه کنید.
          </p>
          <p>
            نمودار مقدار 
            χ2
            بر حسب 
             μ 
            را رسم کنید و آن را تعبیر کنید.
          </p>
    </li>
    <li>
      <h2>
        تمرین ۴
      </h2>
      <p>
        ایا در مقاله ی کشف هیگز میتوانید نموداری پیدا کنید که نتیجه ی شما را تایید کند ؟
      </p>
      <a href="https://arxiv.org/pdf/1207.7235v2.pdf">لینک مقاله</a>
    </li>
  </ul>
</div>